In [6]:
import scipy as sp
import scipy.optimize
import teneva
from time import perf_counter as tpc
from teneva import accuracy_on_data, accuracy, erank
from numba import jit
from opt_einsum import contract

In [7]:
def rand_tensor_idx(I, r):
    d = I.shape[1]
    ns = np.max(I, axis=0) + 1
    return rand_tensor(d, ns, r)


def rand_tensor(d, ns, r):
    ranks = [1] + [r]*(d - 1) + [1]
    
    res = [np.random.rand(*shape) for shape in zip(ranks, ns, ranks[1:])]
        
    return res
    
    
@jit
def _poly_arr(x, n, a=-1, b=1):

    x = -1. + 2.*(x - a)/(b-a)
        
    res = np.ones((len(x), n))
    if n == 1:
        return res
    res[:, 1] = x
    for i in range(2, n):
        res[:, i] = 2*x*res[:, i-1] - res[:, i-2]
        
    return res
        
def poly_arr(x, n, a=-1, b=1):
    x = np.asarray(x)
    reduce_dim = x.ndim == 0
    if reduce_dim:
        x = x[None]
        
    res = _poly_arr(x, n, a=-1, b=1)
    if reduce_dim:
        res = res[0]
        
    return res

    
    
    
    

In [8]:
np.random.seed(42)
h = 30
d = 5
N = 100000
rank = 3

pow_p = 8


train = teneva.sample_lhs([h]*d, N)
test  = teneva.sample_lhs([h]*d, N)

train_x = train.astype(float)/(0.5*h) - 0.5
test_x = test.astype(float)/(0.5*h) - 0.5


Y0 = rand_tensor_idx(train, rank)
teneva.show(Y0)

Y0_spec = rand_tensor(d, [pow_p]*d, rank)
teneva.show(Y0_spec)


Y_train = scipy.optimize.rosen(train_x.T)
Y_test = scipy.optimize.rosen(test_x.T)
Y_train;

TT-tensor     5D : |30| |30| |30| |30| |30|
<rank>  =    3.0 :    \3/  \3/  \3/  \3/
TT-tensor     5D : |8| |8| |8| |8| |8|
<rank>  =    3.0 :   \3/ \3/ \3/ \3/


# Demo spectral

In [9]:
Y_als_sp = teneva.als_spectral(train_x,  Y_train, Y0_spec, lambda x: poly_arr(x, Y0_spec[0].shape[1], a=-1, b=1), log=True, nswp=100)

# pre | time:      0.862 | rank:   3.0 | 
#   1 | time:      4.249 | rank:   3.0 | eps: 1.1e+08 | 
#   2 | time:      6.196 | rank:   3.0 | eps: 1.0e+00 | 
#   3 | time:      8.113 | rank:   3.0 | eps: 1.0e+00 | 
#   4 | time:      9.984 | rank:   3.0 | eps: 6.4e-02 | 
#   5 | time:     11.946 | rank:   3.0 | eps: 2.1e-03 | 
#   6 | time:     13.908 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


# Demo adaptive

In [8]:
Y_als = teneva.als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10, adaptive=True, maxr=5, eps_adap=1e-3)

# pre | time:      0.063 | rank:   3.0 | 
#   1 | time:      3.766 | rank:   5.0 | eps: 1.2e+02 | 
#   2 | time:      7.485 | rank:   5.0 | eps: 2.5e-01 | 
#   3 | time:     11.209 | rank:   5.0 | eps: 8.3e-02 | 
#   4 | time:     14.938 | rank:   5.0 | eps: 3.9e-03 | 
#   5 | time:     18.657 | rank:   4.8 | eps: 1.5e-04 | 
#   6 | time:     22.252 | rank:   3.0 | eps: 1.0e-05 | 
#   7 | time:     25.830 | rank:   3.0 | eps: 7.4e-07 | 
#   8 | time:     29.409 | rank:   3.0 | eps: 2.7e-08 | 
#   9 | time:     32.999 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


# Дальше нерелевантно

In [507]:
def als(I_trn, Y_trn, Y0, nswp=50, e=1.E-16, *, info={}, I_vld=None, Y_vld=None, e_vld=None, 
        log=False, adaptive=False, maxr=20, eps_adap=1e-3):
    """Build TT-tensor by TT-ALS from the given random tensor samples.

    Args:
        I_trn (np.ndarray): multi-indices for the tensor in the form of array
            of the shape [samples, d].
        Y_trn (np.ndarray): values of the tensor for multi-indices I in the form
            of array of the shape [samples].
        Y0 (list): TT-tensor, which is the initial approximation for algorithm.
        nswp (int): number of ALS iterations (sweeps). If "e" or "e_vld"
            parameter is set, then the real number of sweeps may be less (see
            "info" dict with the exact number of performed sweeps).
        e (float): optional algorithm convergence criterion (> 0). If between
            iterations (sweeps) the relative rate of solution change is less
            than this value, then the operation of the algorithm will be
            interrupted.
        info (dict): an optionally set dictionary, which will be filled with
            reference information about the process of the algorithm operation.
            At the end of the function work, it will contain parameters: "e" -
            the final value of the convergence criterion; "e_vld" - the final
            error on the validation dataset; "nswp" - the real number of
            performed iterations (sweeps); "stop" - stop type of the algorithm
            ("nswp", "e" or "e_vld").
        I_vld (np.ndarray): optional multi-indices for items of validation
            dataset in the form of array of the shape [samples, d].
        Y_vld (np.ndarray): optional values for items related to "I_vld" of
            validation dataset in the form of array of the shape [samples].
        e_vld (float): optional algorithm convergence criterion (> 0). If
            after sweep, the error on the validation dataset is less than this
            value, then the operation of the algorithm will be interrupted.
        log (bool): if flag is set, then the information about the progress of
            the algorithm will be printed after each sweep.

    Returns:
        list: TT-tensor, which represents the TT-approximation for the tensor.

    """
    _time = tpc()

    info['r'] = erank(Y0)
    info['e'] = -1.
    info['e_vld'] = -1.
    info['nswp'] = 0
    info['stop'] = None
    
    if eps_adap is None:
        eps_adap = e

    I_trn = np.asanyarray(I_trn, dtype=int)
    Y_trn = np.asanyarray(Y_trn, dtype=float)

    Y = copy(Y0)

    m = I_trn.shape[0]
    d = I_trn.shape[1]

    for k in range(d):
        if np.unique(I_trn[:, k]).size != Y[k].shape[1]:
            raise ValueError('One groundtruth sample is needed for every slice')

    Yl = [np.ones((m, Y[k].shape[0])) for k in range(d)]
    Yr = [np.ones((Y[k].shape[2], m)) for k in range(d)]
            
    for k in range(d-1, 0, -1):
        i = I_trn[:, k]
        Q = Y[k][:, i, :]
        contract('riq,qi->ri', Q, Yr[k], out=Yr[k-1])

    _info(Y, info, _time, I_vld, Y_vld, e_vld, log)
    
    while True:
        Yold = copy(Y)

        for k in range(0, d - 1 - adaptive, +1):
            i = I_trn[:, k]
            if adaptive:
                Y[k], Y[k+1] = _optimize_core_adaptive(Y[k], Y[k+1], i, I_trn[:, k+1], Y_trn, Yl[k], Yr[k+1], maxr=maxr, eps_adap=eps_adap)
                Yl[k+1] = contract('jk,kjl->jl', Yl[k], Y[k][:, i, :])
            else:
                _optimize_core(Y[k], i, Y_trn, Yl[k], Yr[k])
                contract('jk,kjl->jl', Yl[k], Y[k][:, i, :], out=Yl[k+1])

        for k in range(d-1, 0 + adaptive, -1):
            i = I_trn[:, k]
            if adaptive:
                Y[k-1], Y[k] = _optimize_core_adaptive(Y[k-1], Y[k], I_trn[:, k-1], i, Y_trn, Yl[k-1], Yr[k], maxr=maxr, eps_adap=eps_adap)
                Yr[k-1] = contract('ijk,kj->ij', Y[k][:, i, :], Yr[k])
            else:
                _optimize_core(Y[k], i, Y_trn, Yl[k], Yr[k])
                contract('ijk,kj->ij', Y[k][:, i, :], Yr[k], out=Yr[k-1])

        stop = None

        info['e'] = accuracy(Y, Yold)
        if stop is None and info['e'] >= 0 and not np.isinf(info['e']):
            if e is not None and info['e'] <= e:
                stop = 'e'

        info['nswp'] += 1
        if stop is None:
            if nswp is not None and info['nswp'] >= nswp:
                stop = 'nswp'

        if _info(Y, info, _time, I_vld, Y_vld, e_vld, log, stop):
            return Y

        
        
def als_spectral(X_trn, Y_trn, Y0, Hf, nswp=50, e=1.E-16, info={}, I_vld=None, Y_vld=None, e_vld=None, log=False):
    """Build TT-tensor by TT-ALS from the given random tensor samples.

    Args:
        I_trn (np.ndarray): multi-indices for the tensor in the form of array
            of the shape [samples, d].
        Y_trn (np.ndarray): values of the tensor for multi-indices I in the form
            of array of the shape [samples].
        Y0 (list): TT-tensor, which is the initial approximation for algorithm.
        nswp (int): number of ALS iterations (sweeps). If "e" or "e_vld"
            parameter is set, then the real number of sweeps may be less (see
            "info" dict with the exact number of performed sweeps).
        e (float): optional algorithm convergence criterion (> 0). If between
            iterations (sweeps) the relative rate of solution change is less
            than this value, then the operation of the algorithm will be
            interrupted.
        info (dict): an optionally set dictionary, which will be filled with
            reference information about the process of the algorithm operation.
            At the end of the function work, it will contain parameters: "e" -
            the final value of the convergence criterion; "e_vld" - the final
            error on the validation dataset; "nswp" - the real number of
            performed iterations (sweeps); "stop" - stop type of the algorithm
            ("nswp", "e" or "e_vld").
        I_vld (np.ndarray): optional multi-indices for items of validation
            dataset in the form of array of the shape [samples, d].
        Y_vld (np.ndarray): optional values for items related to "I_vld" of
            validation dataset in the form of array of the shape [samples].
        e_vld (float): optional algorithm convergence criterion (> 0). If
            after sweep, the error on the validation dataset is less than this
            value, then the operation of the algorithm will be interrupted.
        log (bool): if flag is set, then the information about the progress of
            the algorithm will be printed after each sweep.

    Returns:
        list: TT-tensor, which represents the TT-approximation for the tensor.

    """
    _time = tpc()

    info['r'] = erank(Y0)
    info['e'] = -1.
    info['e_vld'] = -1.
    info['nswp'] = 0
    info['stop'] = None

    X_trn = np.asanyarray(X_trn, dtype=float)
    Y_trn = np.asanyarray(Y_trn, dtype=float)
    
    Y = copy(Y0)

    m = X_trn.shape[0]
    d = X_trn.shape[1]

    Yl = [np.ones((m, Y[k].shape[0])) for k in range(d)]
    Yr = [np.ones((Y[k].shape[2], m)) for k in range(d)]

    n_shape = Y[0].shape[1] # Assuming they are all the same for now
    
    H = Hf(X_trn.reshape(-1)).reshape((*X_trn.shape, n_shape))
    del X_trn
    
    for k in range(d-1, 0, -1):
        contract('ik,rkq,qi->ri', H[:, k, :], Y[k], Yr[k], out=Yr[k-1])

    _info(Y, info, _time, I_vld, Y_vld, e_vld, log)

    while True:
        Yold = copy(Y)

        for k in range(0, d-1, +1):
            Hk =  H[:, k, :]
            _optimize_core_spec(Y[k],Y_trn, Yl[k], Yr[k], Hk)
            contract('jr,jk,krl->jl', Hk, Yl[k], Y[k], out=Yl[k+1])

        for k in range(d-1, 0, -1):
            Hk =  H[:, k, :]
            _optimize_core_spec(Y[k], Y_trn, Yl[k], Yr[k], Hk)
            contract('jr,irk,kj->ij', Hk, Y[k], Yr[k], out=Yr[k-1])

        stop = None

        info['e'] = accuracy(Y, Yold)
        if stop is None and info['e'] >= 0 and not np.isinf(info['e']):
            if e is not None and info['e'] <= e:
                stop = 'e'

        info['nswp'] += 1
        if stop is None:
            if nswp is not None and info['nswp'] >= nswp:
                stop = 'nswp'

        if _info(Y, info, _time, I_vld, Y_vld, e_vld, log, stop):
            return Y
        

def als_old(I_trn, Y_trn, Y0, nswp=50, e=1.E-16, info={}, I_vld=None, Y_vld=None, e_vld=None, log=False):
    """Build TT-tensor by TT-ALS from the given random tensor samples.

    Args:
        I_trn (np.ndarray): multi-indices for the tensor in the form of array
            of the shape [samples, d].
        Y_trn (np.ndarray): values of the tensor for multi-indices I in the form
            of array of the shape [samples].
        Y0 (list): TT-tensor, which is the initial approximation for algorithm.
        nswp (int): number of ALS iterations (sweeps). If "e" or "e_vld"
            parameter is set, then the real number of sweeps may be less (see
            "info" dict with the exact number of performed sweeps).
        e (float): optional algorithm convergence criterion (> 0). If between
            iterations (sweeps) the relative rate of solution change is less
            than this value, then the operation of the algorithm will be
            interrupted.
        info (dict): an optionally set dictionary, which will be filled with
            reference information about the process of the algorithm operation.
            At the end of the function work, it will contain parameters: "e" -
            the final value of the convergence criterion; "e_vld" - the final
            error on the validation dataset; "nswp" - the real number of
            performed iterations (sweeps); "stop" - stop type of the algorithm
            ("nswp", "e" or "e_vld").
        I_vld (np.ndarray): optional multi-indices for items of validation
            dataset in the form of array of the shape [samples, d].
        Y_vld (np.ndarray): optional values for items related to "I_vld" of
            validation dataset in the form of array of the shape [samples].
        e_vld (float): optional algorithm convergence criterion (> 0). If
            after sweep, the error on the validation dataset is less than this
            value, then the operation of the algorithm will be interrupted.
        log (bool): if flag is set, then the information about the progress of
            the algorithm will be printed after each sweep.

    Returns:
        list: TT-tensor, which represents the TT-approximation for the tensor.

    """
    _time = tpc()

    info['r'] = erank(Y0)
    info['e'] = -1.
    info['e_vld'] = -1.
    info['nswp'] = 0
    info['stop'] = None

    I_trn = np.asanyarray(I_trn, dtype=int)
    Y_trn = np.asanyarray(Y_trn, dtype=float)

    Y = copy(Y0)

    m = I_trn.shape[0]
    d = I_trn.shape[1]

    for k in range(d):
        if np.unique(I_trn[:, k]).size != Y[k].shape[1]:
            raise ValueError('One groundtruth sample is needed for every slice')

    Yl = [np.ones((1, m, Y[k].shape[0])) for k in range(d)]

    Yr = [None for _ in range(d-1)] + [np.ones((1, m, 1))]
    for k in range(d-1, 0, -1):
        i = I_trn[:, k]
        Q = Y[k][:, i, :]
        Yr[k-1] = np.einsum('riq,qis->ris', Q, Yr[k])

    _info(Y, info, _time, I_vld, Y_vld, e_vld, log)

    while True:
        Yold = copy(Y)

        for k in range(0, d-1, +1):
            i = I_trn[:, k]
            Y[k] = _optimize_core(Y[k], i, Y_trn, Yl[k], Yr[k])
            Yl[k+1] = np.einsum('ijk,kjl->ijl', Yl[k], Y[k][:, i, :])

        for k in range(d-1, 0, -1):
            i = I_trn[:, k]
            Y[k] = _optimize_core(Y[k], i, Y_trn, Yl[k], Yr[k])
            Yr[k-1] = np.einsum('ijk,kjl->ijl', Y[k][:, i, :], Yr[k])

        stop = None

        info['e'] = accuracy(Y, Yold)
        if stop is None and info['e'] >= 0 and not np.isinf(info['e']):
            if e is not None and info['e'] <= e:
                stop = 'e'

        info['nswp'] += 1
        if stop is None:
            if nswp is not None and info['nswp'] >= nswp:
                stop = 'nswp'

        if _info(Y, info, _time, I_vld, Y_vld, e_vld, log, stop):
            return Y


        

def _optimize_core(Q, i, Y_trn, Yl, Yr):
    for k in range(Q.shape[1]):
        idx = np.where(i == k)[0]
        lhs = Yr[:, idx].T[:, np.newaxis, :]
        rhs = Yl[idx, :]  [:, :, np.newaxis]
        A = (lhs * rhs).reshape(len(idx), -1)
        Ar = A.shape[1]
        b = Y_trn[idx]
        sol, residuals, rank, s = sp.linalg.lstsq(A, b, 
                                                  overwrite_a=True, 
                                                  overwrite_b=True, 
                                                  lapack_driver='gelsy')
        if rank < Ar:
            print(f"Bad cond in LSTSQ: {rank} < {Ar}")
        Q[:, k, :] = sol.reshape(Q[:, k, :].shape)


def _optimize_core_spec(Q, Y_trn, Yl, Yr, Hk):
        m = Yl.shape[0]
        A = contract("li,ik,ij->ikjl", Yr, Yl, Hk).reshape(m, -1)
        b = Y_trn
        Ar = A.shape[1]
        sol, residuals, rank, s = sp.linalg.lstsq(A, b, 
                                                  overwrite_a=True, 
                                                  overwrite_b=True, 
                                                  lapack_driver='gelsy')
        if rank < Ar:
            print(f"Bad cond in LSTSQ: {rank} < {Ar}")
            
        Q[...] = sol.reshape(Q.shape)

        
def _optimize_core_adaptive(Q1, Q2, i, i2, Y_trn, Yl, Yr, maxr=None, eps_adap=1e-6):
    shape = Q1.shape[0], Q2.shape[2]
    Q = np.empty((Q1.shape[0], Q1.shape[1], Q2.shape[1], Q2.shape[2]))
    for k in range(Q1.shape[1]):
        for k2 in range(Q2.shape[1]):
            #idx = np.where(i == k)[0] & np.where(i2 == k2)[0]
            idx = (i == k) & (i2 == k2)
            assert idx.any(), 'Non enouph samples' # Add this check to the main func at the beggining

            lhs = Yr[:, idx].T[:, np.newaxis, :]
            rhs = Yl[idx, :]  [:, :, np.newaxis]
            A = (lhs * rhs).reshape(idx.sum(), -1)
            Ar = A.shape[1]
            b = Y_trn[idx]
            sol, residuals, rank, s = sp.linalg.lstsq(A, b, 
                                                      overwrite_a=True, 
                                                      overwrite_b=True, 
                                                      lapack_driver='gelsy')
            if rank < Ar:
                print(f"Bad cond in LSTSQ: {rank} < {Ar}")
            Q[:, k, k2, :] = sol.reshape(shape)
            
    Q = Q.reshape(np.prod(Q.shape[:2]), -1)
    V1, V2 = matrix_skeleton(Q, r=maxr, e=eps_adap, rel=True)
    return V1.reshape(*Q1.shape[:2], -1), V2.reshape(-1, *Q2.shape[1:])
        
    
def matrix_skeleton(A, e=1.E-10, r=1.E+12, hermitian=False, rel=False):
    """Construct truncated skeleton decomposition A = U V for the given matrix.

    Args:
        A (np.ndarray): matrix of the shape [m, n].
        e (float): desired approximation accuracy (> 0).
        r (int, float): maximum rank for the SVD decomposition (> 0).
        hermitian (flag): if True, then "hermitian" SVD will be used.

    Returns:
        [np.ndarray, np.ndarray]: factor matrix U of the shape [m, q] and factor
        matrix V of the shape [q, n], where "q" is selected rank (q <= r).

    """
    U, s, V = np.linalg.svd(A, full_matrices=False, hermitian=hermitian)
    if rel:
        r = max(1, min(int(r), sum(s/s[0] > e)))
    else:
        r = max(1, min(int(r), sum(s > e)))
            
    S = np.diag(np.sqrt(s[:r]))
    return U[:, :r] @ S, S @ V[:r, :]


In [511]:
!which python

~/anaconda2/envs/py36/bin/python


In [431]:


def _info(Y, info, t, I_vld, Y_vld, e_vld, log=False, stop=None):
    info['e_vld'] = accuracy_on_data(Y, I_vld, Y_vld)

    if stop is None and info['e_vld'] >= 0 and not np.isinf(info['e_vld']):
        if e_vld is not None and info['e_vld'] <= e_vld:
            stop = 'e_vld'
    info['stop'] = stop

    info['r'] = erank(Y)
    info['t'] = tpc() - t

    _log(Y, info, log)

    return info['stop']


def _log(Y, info, log):
    if not log:
        return

    text = ''

    if info['nswp'] == 0:
        text += f'# pre | '
    else:
        text += f'# {info["nswp"]:-3d} | '

    text += f'time: {info["t"]:-10.3f} | '

    text += f'rank: {info["r"]:-5.1f} | '

    if info['e_vld'] >= 0:
        text += f'err: {info["e_vld"]:-7.1e} | '

    if info['e'] >= 0:
        text += f'eps: {info["e"]:-7.1e} | '

    if info['stop']:
        text += f'stop: {info["stop"]} | '

    print(text)



def copy(Y):
    """Return a copy of the given TT-tensor.

    Args:
        Y (int, float, list): TT-tensor (or it may be int/float).

    Returns:
        list: TT-tensor, which is a copy of the given TT-tensor. If Y is a
        number, then result will be the same number. If Y is np.ndarray, then
        the result will the corresponding copy in numpy format.

    """
    if _is_num(Y):
        return Y
    elif isinstance(Y, np.ndarray):
        return Y.copy()
    else:
        return [G.copy() for G in Y]
    
def _is_num(A):
    return isinstance(A, (int, float))




In [502]:
Y_als_sp = als_spectral(train_x,  Y_train, Y0_spec, lambda x: poly_arr(x, Y0_spec[0].shape[1], a=-1, b=1), log=True, nswp=100)

# pre | time:      0.082 | rank:   3.0 | 
#   1 | time:      2.143 | rank:   3.0 | eps: 1.1e+08 | 
#   2 | time:      4.046 | rank:   3.0 | eps: 1.0e+00 | 
#   3 | time:      6.087 | rank:   3.0 | eps: 1.0e+00 | 
#   4 | time:      7.929 | rank:   3.0 | eps: 6.4e-02 | 
#   5 | time:      9.791 | rank:   3.0 | eps: 2.1e-03 | 
#   6 | time:     11.698 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


In [400]:
train_x;

In [429]:
Y_als_sp[1]


array([[[ 5.0538269570e+03, -4.4028972898e+03, -3.6335756338e+02],
        [-4.5443186427e+03,  4.1854380358e+03,  9.6032635919e+01],
        [ 1.8294720097e+03, -3.5971013082e+02, -1.0663748303e+03],
        [ 4.9775722720e-04, -3.1466939565e-04, -1.4673316688e-04],
        [ 2.8929418011e+02, -2.6644765627e+02, -6.1134552863e+00],
        [ 1.1389822358e-04, -9.3466894069e-05, -1.7285449819e-05],
        [-3.0282662009e-05,  3.1610129949e-05, -5.2744781996e-07],
        [ 3.3183212291e-06, -4.6514015596e-06,  9.5360670022e-07]],

       [[ 5.6011000254e+03, -4.8931980883e+03, -3.8893421596e+02],
        [-4.6106746223e+03,  4.2465536268e+03,  9.7434925563e+01],
        [ 1.9293850525e+03, -3.7935503191e+02, -1.1246127880e+03],
        [ 4.6603694019e-04, -2.9551680120e-04, -1.3655036366e-04],
        [ 3.0509343467e+02, -2.8099918356e+02, -6.4473376491e+00],
        [ 1.0696964105e-04, -8.7863877875e-05, -1.6128252388e-05],
        [-2.8532706226e-05,  2.9740746367e-05, -4.7804384350

In [509]:
Y_als = als(train,  Y_train, Y0, log=True, e=1e-10, nswp=10, adaptive=True, maxr=5, eps_adap=1e-3)

# pre | time:      0.032 | rank:   3.0 | 
#   1 | time:      3.758 | rank:   5.0 | eps: 1.2e+02 | 
#   2 | time:      7.447 | rank:   5.0 | eps: 2.5e-01 | 
#   3 | time:     11.139 | rank:   5.0 | eps: 8.3e-02 | 
#   4 | time:     14.836 | rank:   5.0 | eps: 3.9e-03 | 
#   5 | time:     18.526 | rank:   4.8 | eps: 1.5e-04 | 
#   6 | time:     22.091 | rank:   3.0 | eps: 1.0e-05 | 
#   7 | time:     25.635 | rank:   3.0 | eps: 7.4e-07 | 
#   8 | time:     29.175 | rank:   3.0 | eps: 2.7e-08 | 
#   9 | time:     32.725 | rank:   3.0 | eps: 0.0e+00 | stop: e | 


In [299]:
Y_als = teneva.als(train,  Y_train, Y0, log=True, nswp=30)

# pre | time:      0.001 | rank:   3.0 | 
#   1 | time:      0.029 | rank:   3.0 | eps: 2.1e+02 | 
#   2 | time:      0.057 | rank:   3.0 | eps: 4.2e-01 | 
#   3 | time:      0.075 | rank:   3.0 | eps: 2.5e-01 | 
#   4 | time:      0.094 | rank:   3.0 | eps: 1.6e-01 | 
#   5 | time:      0.112 | rank:   3.0 | eps: 1.1e-01 | 
#   6 | time:      0.131 | rank:   3.0 | eps: 8.9e-02 | 
#   7 | time:      0.149 | rank:   3.0 | eps: 7.8e-02 | 
#   8 | time:      0.168 | rank:   3.0 | eps: 7.0e-02 | 
#   9 | time:      0.186 | rank:   3.0 | eps: 6.4e-02 | 
#  10 | time:      0.205 | rank:   3.0 | eps: 5.9e-02 | 
#  11 | time:      0.224 | rank:   3.0 | eps: 5.5e-02 | 
#  12 | time:      0.242 | rank:   3.0 | eps: 5.2e-02 | 
#  13 | time:      0.261 | rank:   3.0 | eps: 5.0e-02 | 
#  14 | time:      0.279 | rank:   3.0 | eps: 4.7e-02 | 
#  15 | time:      0.298 | rank:   3.0 | eps: 4.2e-02 | 
#  16 | time:      0.316 | rank:   3.0 | eps: 3.8e-02 | 
#  17 | time:      0.334 | rank:   3.0 | eps: 

In [492]:
Y_als[1];

In [510]:
accuracy_on_data(Y_als, train, Y_train), accuracy_on_data(Y_als, test, Y_test)

(2.0847552792748135e-12, 2.123440281771813e-12)

In [162]:
train.shape, len(Y0)

((1000, 5), 5)

In [452]:
np.where(np.arange(10) > 4)

(array([5, 6, 7, 8, 9]),)